## Testando Predição

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


In [2]:
# Imports
import pandas as pd
import pickle
import warnings
import tools
from datetime import datetime as dt
warnings.filterwarnings("ignore")

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Fabiano Fonseca - MLE" --iversions

Author: Fabiano Fonseca - MLE

pandas: 1.3.5



### Carregando dataset de test

In [37]:
csvFileName = "../data/test.csv"
df_test = pd.read_csv(csvFileName, parse_dates=["pickup_datetime"])

In [38]:
df = df_test.copy()

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                9914 non-null   object             
 1   pickup_datetime    9914 non-null   datetime64[ns, UTC]
 2   pickup_longitude   9914 non-null   float64            
 3   pickup_latitude    9914 non-null   float64            
 4   dropoff_longitude  9914 non-null   float64            
 5   dropoff_latitude   9914 non-null   float64            
 6   passenger_count    9914 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 542.3+ KB


In [29]:
# Shape
df.shape

(9914, 7)

In [43]:
df.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,timestamp,hour,day_of_week,...,day_of_week_norm,cos_day_of_week,sin_day_of_week,day_of_month_norm,cos_day_of_month,sin_day_of_month,week_norm,cos_week,sin_week,distance_km
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27 13:08:24,13,1,...,1.047198,0.5,0.866025,5.472452,0.688967,-0.724793,0.592753,0.829406,0.558647,2.3
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27 13:08:24,13,1,...,1.047198,0.5,0.866025,5.472452,0.688967,-0.724793,0.592753,0.829406,0.558647,2.4
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08 11:53:44,11,5,...,5.235988,0.5,-0.866025,1.621467,-0.050649,0.998717,4.742027,0.029633,-0.999561,0.6
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01 21:12:12,21,5,...,5.235988,0.5,-0.866025,0.202683,0.979530,0.201299,5.690432,0.829406,-0.558647,2.0
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01 21:12:12,21,5,...,5.235988,0.5,-0.866025,0.202683,0.979530,0.201299,5.690432,0.829406,-0.558647,5.4


In [44]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1


In [45]:
# não vai haver remoção de linhas

# transformando a feature datetime
df = tools.split_datetime(df)
# convertendo para features cíclicas
df = tools.cyclic_features(df)
df = tools.apply_distance(df)

In [46]:
df = reduce_size_features(df, verbose=False)

In [47]:
# remove features não necessárias
df.drop(['key','pickup_datetime','timestamp'],axis=1, inplace=True)

### Carrega o modelo serializado e faz predição

In [48]:
modelo = pickle.load(open('../modelo/modelo_20220321.sav','rb'))

In [49]:
# Acrescenta o resultado da predição do dataset de test
df_test['pred_fare_amount'] = modelo.predict(df)

In [51]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pred_fare_amount
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1,9.196494
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1,9.690596
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1,5.291669
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1,8.865063
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1,15.083334


In [52]:
# salva dataset com valor achado de predição ao datset de test
# path
csvFileName = "../data/test_pred_batch_" + "_" + dt.now().strftime("%Y%m%d") + ".csv"
df_test.to_csv(csvFileName);

### Fim